<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0205minnichi_lm_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- date: 2022_0206
- filename: 2022_0205minnichi_lm_generator.ipynb

# [「みんなの日本語」](https://www.3anet.co.jp/np/books/2300/) 語彙を用いた [符号化ー復号化 (encoder-decoder) モデル](https://arxiv.org/pdf/1409.3215) による言語生成

- 想定発表媒体: 某日本語教育研究会論文誌
- 著者: 岩下 智彦，吉原 将大，浅川伸一


### かなり無駄な薀蓄:

このファイルを自分のローカル PC で動作させるためには，Python の処理系とブラウザベースの統合実行環境，たとえば ```jupyter-notebook``` または ```jupyter-lab``` が必要になります。
OS が MacOSX であれば，パッケージ管理 [homebrew](https://brew.sh/) と Python ライブラリ管理 [anaconda](https://www.anaconda.com/products/individual) がインストール済であれば便利です。
加えて，`PyTorch`, `MeCab`, `jaconv`, `japanize_matplotlib`, `konoha`, `sacrebleu` と言った，ライブラリをインストールしておく必要があります。
以下に上記 4 つのライブラリのインスールを行うサンプルオペレーションを示します。

```bash
conda install pytorch torchvision torchaudio -c pytorch

brew install mecab
brew install mecab-ipadic
pip install mecab-python3

pip install jaconv
pip install japanize-matplotlib
pip install 'konoha[mecab]'
pip install 'sacrebleu[ja]'
```

ローカル Mac に自力で，環境構築をする場合の老婆(翁)心的基本方針を記して起きます。
homebrew にパッケージ管理を極力任せる方が，後々の管理が楽になるでしょう。
自力で複数のパッケージを管理すると，依存関係が煩雑になって心折れてしまいます。
Homebrew が対応していない場合のみ，anaconda 付属の pip でイントールするという方針で行くと，自分のミスや誤解に起因するインストール済ライブラリの依存関係の不一致に悩む可能性が減ります。

本コードをローカル Mac で動作させるためには，git コマンドを用いて Github 上にアップロードした自作クラス `Minnichi` をローカルディスクに保存する必要があります。
これは直下セルの 12 行目で実行しているコマンドを，自身のローカル Mac 上で行うことを意味します。
具体的には，Mac の「端末エミュレータ」あるいは類似のエミュレータ上で以下のコマンドを実行します。

```bash
git clone https://github.com/ShinAsakawa/ccap.git
```

こうすることで，カレントディレクトリ直下に `ccap` というディレクトリが作成されます。
この `ccap` ディレクトリ内に `minnichi.py` というファイルがあります。
`ccap` とは，本プロジェクトとは異なるプロジェクトです。
もちろん，将来的には，別の github レポジトリを作成した方が良いと考えています。
ですが，当面のお試しコードの意味合いもありますので，今回は `ccap` プロジェクトのレポジトリに間借りして作成してあります。


# 0 準備: 必要となるライブラリのインストールなど

In [ ]:
import os
import sys
import numpy as np
import random
from termcolor import colored

# ローカルと colab との相違を吸収するために
# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストール
import platform
isColab = platform.system() == 'Linux'
if isColab:
    ![ -d ccap ] & /bin/rm -rf ccap
    !git clone https://github.com/ShinAsakawa/ccap.git
    !pip install japanize_matplotlib > /dev/null 2>&1
    !pip install jaconv > /dev/null 2>&1

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_ipadic = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                    shell=True).communicate()[0]).decode('utf-8')

    !pip install 'konoha[mecab]'
    !pip install 'sacrebleu[ja]'
    #!pip install 'fugashi[unidic]' > /dev/null 2>&1
    #!python -m unidic download > /dev/null 2>&1
    #!pip install ipadic > /dev/null 2>&1

# 1「みんなの日本語」語彙データセットの読み込み

In [ ]:
%reload_ext autoreload
%autoreload 2

import MeCab
import jaconv
from ccap.minnichi import Minnichi

Minn = Minnichi(reload=True)
#Minn.save_data()
Minn = Minnichi(reload=False)

print(Minn.tokenize('お前はトラだ。虎になるのだ----', max_length=25,pad=False)['input_ids'])
print(Minn.convert_ids2tokens(Minn.tokenize('お前はトラだ。虎になるのだ----', max_length=20,pad=False)['input_ids']))

In [ ]:
for i in range(3):
    print(Minn.lines[i])


## 1.1 語彙頻度の描画

In [ ]:
Minn.draw_freq(figsize=(28,8),rotation=25)
MAX_LENGTH = Minn.max_length + 1
#Minn(1)

# 2 学習に用いる，符号化器 (エンコーダ)，復号化器 (デコーダ) の定義

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#データセットのためのクラスを定義
class minnichiDataset(torch.utils.data.Dataset):
    def __init__(self, encoder:Minnichi):
        self.encoder = encoder
        
    def __getitem__(self, idx):
        return self.encoder(idx)['input_ids']
    
    def __len__(self):
        return self.encoder.__len__()


class EncoderRNN(nn.Module):
    """RNNによる符号化器"""
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoderRNN(nn.Module):
    """注意付き復号化器の定義"""
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 3 訓練関数 `train()` の定義 教師強制付き

In [ ]:
def tensorFromIds(sentence_ids):
    return torch.tensor(sentence_ids, dtype=torch.long, device=device).view(-1, 1)

teacher_forcing_ratio = 0.5  # 訳注：教師強制率。文献によっては，訓練中にこの値を徐々に減衰させることも行われます

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden() # 符号化器の中間層を初期化
    encoder_optimizer.zero_grad()         # 符号化器の最適化関数の初期化
    decoder_optimizer.zero_grad()         # 復号化器の最適化関数の初期化

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[Minn.vocab.index('<SOS>')]], device=device)
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            #if decoder_input.item() == EOS_token:
            if decoder_input.item() == Minn.vocab.index('<EOS>'):
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math

def asMinutes(s):
    """時間変数を見やすいように，分と秒に変換して返す"""
    m = math.floor(s / 60)
    s -= m * 60
    return f'{int(m):2d}分 {int(s):2d}秒'
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    """開始時刻 since と，現在の処理が全処理中に示す割合 percent を与えて，経過時間と残り時間を計算して表示する"""
    now = time.time()  #現在時刻を取得
    s = now - since    # 開始時刻から現在までの経過時間を計算
    #s = since - now    
    es = s / (percent) # 経過時間を現在までの処理割合で割って終了予想時間を計算
    rs = es - s        # 終了予想時刻から経過した時間を引いて残り時間を計算
    #return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
    return f'経過時間:{asMinutes(s)} (残り時間 {asMinutes(rs)})'

# 4 `fit()` 関数の定義 エポックを反復して `train()` を呼び出す

下記のセル 11, 12 行目のコメントアウトを解除し，かつ，13, 14 行目をコメントアウトすると，学習時のアルゴリズムが変わる。

SGD は 確率的勾配降下法 (Bottou, 2003 など) であり，[Adam](https://arxiv.org/pdf/1412.6980/) に比べれば低速である。
採択するアルゴリズムによって，どのような値に落ち着くのか，反復回数だけでは説明できない収束状況が考えられる。
このことから，学習回数と語彙獲得とを並列しては語れないことの傍証になっていると思うのだが，いかがだろうか。

In [ ]:
def fit(encoder:nn.Module, 
        decoder:nn.Module, 
        epochs:int=20, 
        lr:float=0.001, 
        n_sample:int=3)->list:
    
    start_time = time.time()
    
    encoder.train()
    decoder.train()
    #encoder_optimizer = optim.SGD(encoder.parameters(), lr=lr)
    #decoder_optimizer = optim.SGD(decoder.parameters(), lr=lr)
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)
    criterion = nn.NLLLoss()
    losses = []

    for epoch in range(epochs):
        epoch_loss = 0
        
        #エポックごとに学習順をシャッフルする
        learning_order = np.random.permutation(len(Minn.lines)) 
        for i in range(len(Minn.lines)):
            x = learning_order[i]   # ランダムにデータを取り出す 
            inputs = Minn(x)['input_ids']
            input_tensor = tensorFromIds(inputs)
            target_tensor = tensorFromIds(inputs)
            
            #訓練の実施
            loss = train(input_tensor, target_tensor, 
                         encoder, decoder, 
                         encoder_optimizer, decoder_optimizer, 
                         criterion)
            epoch_loss += loss
        
        losses.append(epoch_loss/len(Minn.vocab))
        print(colored(f'エポック:{epoch:2d} 損失:{epoch_loss/len(Minn.vocab):.2f}', 'cyan', attrs=['bold']),
              f'{timeSince(start_time, (epoch+1) * len(Minn.vocab)/(epochs * len(Minn.vocab)))}')
        
        evaluateRandomly(encoder,decoder, n=n_sample)
        
    return losses

# 5 評価関数 `evaluate()` の定義

In [ ]:
def evaluate(encoder:nn.Module, 
             decoder:nn.Module, 
             input_ids:list, 
             max_length:int=MAX_LENGTH)->(list,torch.LongTensor):
    with torch.no_grad():
        input_tensor = tensorFromIds(input_ids)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[Minn.vocab.index('<SOS>')]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == Minn.vocab.index('<EOS>'):
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(Minn.vocab[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
from sacrebleu import corpus_bleu

def calc_bleu(predictions:list, references:list)->float:
    references = [references]
    bleu_score = corpus_bleu(predictions, references,      
                             smooth_method="exp",
                             smooth_value=0.0,
                             force=False,
                             lowercase=False,
                             tokenize="ja-mecab",
                             use_effective_order=False)
    return bleu_score.score

In [ ]:
def evaluateRandomly(encoder:nn.Module, 
                     decoder:nn.Module, 
                     n:int=5)->float:
    
    srcs, preds = [], []
    for x in np.random.randint(Minn.__len__(), size=n):
        input_ids = Minn(x)['input_ids']
        input_sent = "".join(Minn(x)['tokens'])
        print(f'入力: {input_ids}: {input_sent}')
        output_words, attentions = evaluate(encoder, decoder, input_ids)
        output_sent = "".join(w for w in output_words)

        srcs.append(input_sent)
        preds.append(output_sent)
        print(f'出力: {[Minn.vocab.index(c) for c in output_words]}',
              f':{output_sent}')
        print('---')
    return bleu(srcs, preds)

# 6 学習の実施

In [ ]:
%%time
hidden_size = 256
encoder = EncoderRNN(len(Minn.vocab), hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, len(Minn.vocab), dropout_p=0.1).to(device)

losses = []
losses = losses + fit(encoder, decoder, epochs=10, n_sample=2)

In [ ]:
%%time
#losses = []
losses = losses + fit(encoder, decoder, epochs=3, n_sample=2)

# 7 学習経過の描画

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points:list)->None:
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    
showPlot(losses)    

In [ ]:
print(f'BLUE スコア: {evaluateRandomly(encoder, decoder, n=5):.2f}')

# 8 自由入力文による評価

In [ ]:
def evaluate_free_input(encoder:nn.Module, 
                        decoder:nn.Module,
                        inp=None,
                       )->None:
    if inp == None:
        inp = input()
    inp = jaconv.normalize(inp)
    inputs = Minn.tokenize(inp, pad=False)
    input_ids = inputs['input_ids']
    #input_sent = "".join(inputs['tokens'])
    #print(f'入力: {input_ids}: {input_sent}')
    output_tokens, attentions = evaluate(encoder, decoder, input_ids)
    output_ids = Minn.convert_tokens2ids(output_tokens)
    return input_ids, output_ids


In [ ]:
inp, out = evaluate_free_input(encoder,decoder, inp='お前は虎だ。虎になるのだ。')
print(Minn.convert_ids2tokens(inp))
print(Minn.convert_ids2tokens(out))

# 9 エンコーダの内部表現の取得

In [ ]:
def get_an_encoder_representation(encoder:nn.Module,
                                  input_ids:list,
                                  max_length:int=MAX_LENGTH)->(list,torch.LongTensor):
    with torch.no_grad():
        input_tensor = tensorFromIds(input_ids)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        return encoder_hidden


In [ ]:
X = np.zeros((Minn.__len__(),256))
for i in range(Minn.__len__()):
    x = get_an_encoder_representation(encoder, input_ids = Minn(1)['input_ids'])
    X[i] = x.squeeze(0).clone().detach().numpy()[0]

print(X.shape)
#Minn(1)['input_ids']

In [ ]:
import ccap.tsne as tsne
#help(tsne)
tsne_result = tsne.tsne(X)

# 10 内部表現の描画

In [ ]:
tsne_result.shape
plt.figure(figsize=(10,10))
plt.scatter(tsne_result[:,0],tsne_result[:,1])
plt.show()

In [ ]:
x_min = tsne_result[:,0].argmin()
x_max = tsne_result[:,0].argmax()
y_min = tsne_result[:,1].argmin()
y_max = tsne_result[:,1].argmax()

print(f'X 座標最小値:{Minn(x_min)["tokens"]}')
print(f'X 座標最大値:{Minn(x_max)["tokens"]}')
print(f'Y 座標最小値:{Minn(y_min)["tokens"]}')
print(f'Y 座標最大値:{Minn(y_max)["tokens"]}')


# 11 文章生成のアイデア

## 11.1 2 文の内挿による文生成

## 11.2 コアとなる文章に乱数を付加した文生成




## 11.3 国立国語研究所 日本語均衡コーパスによる検証

In [ ]:
with open('ccap/2022_0206bccwj_minnichi_filtered.txt', 'r', encoding='utf8') as f:
    b = f.readlines()
bccwj = [_b.strip() for _b in b]

In [ ]:
X = np.random.randint(len(bccwj), size=3)
preds, refs = [], []
for x in X:
    l = bccwj[x]
    ref, pred = evaluate_free_input(encoder, decoder, inp=l)
    ref_s = "".join(w for w in Minn.convert_ids2tokens(ref)).replace('<EOS>','')
    pred_s = "".join(w for w in Minn.convert_ids2tokens(pred)).replace('<EOS>','')
                       
    preds.append(pred_s)
    refs.append(ref_s)
    print(f'入力:{ref_s}')
    print(f'出力:{pred_s}')
    print('---')
    
print(f'BLEU スコア:{bleu(preds, refs):.2f}')